<a href="https://colab.research.google.com/github/michaelnikhil/my_python_monitor_covid19/blob/master/my_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID19 monitor

Python notebook to display statistics on the virus cases/deaths

Data from John Hopkins University, Center of System Science and Engineering
https://github.com/CSSEGISandData


Imports and class to load the data




In [0]:
import pandas as pd
import re
import datetime
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt

class LoadData():
    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            df = pd.read_csv(self.url, sep=',', na_values=['nan'], na_filter=True)
        except:
            # default local file
            df = pd.read_csv('time_series_covid19_deaths_global.csv', sep=',', na_values=['nan'], na_filter=True)
        return df

    def retrieve_dates(self, df):
        unformatted_dates = df.columns[4:len(df.columns)]
        unformatted_dates2 = [re.sub(r"\d{2}$", r"2020", x) for x in unformatted_dates]
        dates = [datetime.datetime.strptime(x, '%m/%d/%Y') for x in unformatted_dates2]
        return dates

    def sort10(self, df):
        tmp = df.sort_values(df.columns[-1], ascending=False)
        lastDate = df.columns[-1]
        return tmp['Country/Region'][0:10], tmp[tmp.columns[-1]][0:10], lastDate

covid19_class : class Country

In [0]:
class Country():
    def __init__(self, df, country_name, dates):
        self.df = df
        self.country_name = country_name

    def nvalues(self):
        tmp = self.df[self.df['Country/Region'] == self.country_name]
        tmp = tmp.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1)
        val = []
        for col in tmp:
            val.append(sum(tmp[col]))
        return val

Utility functions


In [0]:
def daily_values(cumul):
    val = [0]
    for i in range(1, len(cumul)):
        val.append(cumul[i] - cumul[i - 1])
    return val


def format_plot(fig1, ax0):
    ax0.legend(fontsize=12, loc=0)
    ax0.xaxis.set_major_formatter(mdates.DateFormatter('%d%b'))
    ax0.yaxis.grid(True)
    fig1.autofmt_xdate()

main script to collect the data

In [0]:
# url of the data in csv file
url1 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
url2 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

# choose the countries to analyse
list_of_countries = ['Italy', 'Spain', 'United Kingdom', 'France', 'US', 'Sweden']

# import all data from the url
death = LoadData(url1)
data_death = death.download()
conf = LoadData(url2)
data_conf = conf.download()
dates = death.retrieve_dates(data_death)

# summary 10 biggest values
country_sort10, death_sort10, lastDate = death.sort10(data_death)

plots

In [0]:
    # plot values per country and comparing between countries
    fig1, ax0 = plt.subplots(figsize=(8, 4))
    pos = np.arange(10)
    ax0.bar(pos, death_sort10, align='center')
    ax0.set_ylabel('deaths')
    plt.xticks(pos, country_sort10)
    ax0.set_title(lastDate)
    fig1.autofmt_xdate()

    fig10, (ax10, ax11) = plt.subplots(ncols=2, figsize=(10, 6))
    for country in list_of_countries:
    # number of deaths
      death_country = Country(data_death, country, dates)
      n_death_country = Country(data_death, country, dates).nvalues()
      n_death_daily = daily_values(n_death_country)
      # number of confirmed cases
      conf_country = Country(data_conf, country, dates)
      n_conf_country = Country(data_conf, country, dates).nvalues()
      n_conf_daily = daily_values(n_conf_country)

      # plot per country
      fig1, (ax0, ax1) = plt.subplots(nrows=2, figsize=(8, 6))
      ax0.plot(dates, n_death_country, label='deaths')
      # ax0.plot(dates,n_conf_country,label='confirmed cases')
      ax0.set_ylabel('total', fontsize=12)
      ax0.set_title(country)
      format_plot(fig1, ax0)

      ax1.plot(dates, n_death_daily, label='deaths')
      # ax1.plot(dates,n_conf_daily,label='confirmed cases')
      ax1.set_ylabel('daily', fontsize=12)
      format_plot(fig1, ax1)

      # compare countries
      ax10.plot(dates, n_death_country, label=country)
      ax10.set_ylabel('death', fontsize=12)
      format_plot(fig10, ax10)

      ax11.semilogy(dates, n_death_country, label=country)
      ax11.set_ylabel('death', fontsize=12)
      format_plot(fig10, ax11)